# Jaime Ariel Chitay
# 19002065 


# Descripcion del dataset:

Se buscará encontrar un modelo para la resistencia a la compresion del concreto, este Dataset, contiene 6 variables de los ingredientes del concreto y la edad. Por tanto tendremos un dataset con 7 variables de entrada y una de salida. El modelo busca predecir un modelo de regresión.

Elementos para encontrar la resistencia a la compresión del concreto:

1. cemento
2. escoria de alto horno
3. cenizas volantes
4. agua
5. superplastificante
6. agregado grueso 
7. agregado fino.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

In [3]:
dataset = pd.read_excel('Concrete_Data.xls', index_col=0)
dataset.shape
dataset.head(20)

,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age,Concrete_compressive_strength
Cement,,,,,,,,
540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.029847
380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.698299
380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.447770
266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.854291


# Preprocesamiento de la data.

Se tomarán los siguientes aspectos: 

* Deep Learning no necesita feature engineering.
* No hay que aplicar one-hot encoding por el tipo de variables trabajadas. 

### Normalizacion de las features: 

Formula: 

**x' = (x - xmin)/(xmax - xmin)**



In [4]:
scaling = preprocessing.MinMaxScaler()
scaledMatrix = scaling.fit_transform(dataset)
df_normalized = pd.DataFrame(scaledMatrix,columns=dataset.columns)
df_normalized.head(10)

,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age,Concrete_compressive_strength
0,0.000000,0.0,0.321357,0.07764,0.694767,0.205720,0.074176,0.967445
1,0.000000,0.0,0.321357,0.07764,0.738372,0.205720,0.074176,0.741964
2,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,0.739011,0.472642
3,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,1.000000,0.482400
4,0.368392,0.0,0.560878,0.00000,0.515698,0.580783,0.986264,0.522806
5,0.317195,0.0,0.848303,0.00000,0.380814,0.190667,0.244505,0.556864
6,0.264329,0.0,0.848303,0.00000,0.380814,0.000000,1.000000,0.515358
7,0.264329,0.0,0.848303,0.00000,0.380814,0.000000,0.074176,0.425029
8,0.317195,0.0,0.848303,0.00000,0.380814,0.190667,0.074176,0.542219
9,0.000000,0.0,0.848303,0.00000,0.380814,0.000000,0.074176,0.460436


**Separacion de variables dependientes e independientes del modelo**

In [5]:
X = df_normalized[['Blast_Furnace_Slag', 'Fly_Ash', 'Water', 'Superplasticizer', 'Coarse_Aggregate', 'Fine_Aggregate', 'Age',]]
y = df_normalized[['Concrete_compressive_strength']]

In [6]:
X[0:20]

,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age
0,0.000000,0.0,0.321357,0.07764,0.694767,0.205720,0.074176
1,0.000000,0.0,0.321357,0.07764,0.738372,0.205720,0.074176
2,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,0.739011
3,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,1.000000
4,0.368392,0.0,0.560878,0.00000,0.515698,0.580783,0.986264
5,0.317195,0.0,0.848303,0.00000,0.380814,0.190667,0.244505
6,0.264329,0.0,0.848303,0.00000,0.380814,0.000000,1.000000
7,0.264329,0.0,0.848303,0.00000,0.380814,0.000000,0.074176
8,0.317195,0.0,0.848303,0.00000,0.380814,0.190667,0.074176
9,0.000000,0.0,0.848303,0.00000,0.380814,0.000000,0.074176


In [7]:
y[0:20]

,Concrete_compressive_strength
0,0.967445
1,0.741964
2,0.472642
3,0.482400
4,0.522806
5,0.556864
6,0.515358
7,0.425029
8,0.542219
9,0.460436


# Cross validations and k-folds

Se tomaran un 15% de datos para el Test, y un 85% de datos para el Train. Tambien utilizamos crossValidation k folds.

In [8]:
def crossValidKfolds(X,y,k=3,itTest = 3):
    if itTest>k:
        return None
    elif itTest == k:
        return (X[(X.shape[0]//k)*(itTest-1):],   #xTest             
                y[(y.shape[0]//k)*(itTest-1):],   #yTest
                X[:(X.shape[0]//k)*(itTest-1)],   #xTrain
                y[:(y.shape[0]//k)*(itTest-1)])   #yTrain
    elif itTest == 1:
        return (X[(X.shape[0]//k)*(itTest-1): (X.shape[0]//k)*(itTest)],                
                y[(y.shape[0]//k)*(itTest-1): (y.shape[0]//k)*(itTest)],
                X[(X.shape[0]//k)*(itTest):],
                y[(y.shape[0]//k)*(itTest):])
    else:
        firstX = X[:(X.shape[0]//k)*(itTest-1)]
        secndX = X[(X.shape[0]//k)*(itTest):]
        firstY = y[:(y.shape[0]//k)*(itTest-1)]
        secndY = y[(y.shape[0]//k)*(itTest):]
        return (X[(X.shape[0]//k)*(itTest-1): (X.shape[0]//k)*(itTest)],                
                y[(y.shape[0]//k)*(itTest-1): (y.shape[0]//k)*(itTest)],
                pd.concat([firstX, secndX]),
                pd.concat([firstY, secndY])) 

In [9]:
porcentajeTest = 0.15

In [10]:
#Devolvemos el segmento 2 de 6 (int(1/0.15))
xTest01,yTest01,xTrain01,yTrain01 = crossValidKfolds(X,y,k=int(1/porcentajeTest),itTest = 2)
#Devolvemos el segmento 4 de 6 (int(1/0.15))
xTest02,yTest02,xTrain02,yTrain02 = crossValidKfolds(X,y,k=int(1/porcentajeTest),itTest = 4)
#Devolvemos el segmento 6 de 6 (int(1/0.15))
xTest03,yTest03,xTrain03,yTrain03 = crossValidKfolds(X,y,k=int(1/porcentajeTest),itTest = 6)

**Agregando el Bias a cada set de entrenamiento**

In [12]:
print(xTest01.shape)
xTest01.head()

(171, 7)


,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age
171,0.000000,0.0,0.160080,0.875776,0.148547,0.751882,0.247253
172,0.295771,0.0,0.253493,0.512422,0.148547,0.735324,0.247253
173,0.591263,0.0,0.271058,0.444099,0.148547,0.718515,0.247253
174,0.263495,0.0,0.204790,0.354037,0.423837,0.647516,0.247253
175,0.525876,0.0,0.344511,0.360248,0.417733,0.405921,0.247253


In [13]:
print(yTest01.shape)
yTest01.head()

(171, 1)


,Concrete_compressive_strength
171,0.708464
172,0.783195
173,0.819358
174,0.911526
175,0.958855


In [15]:
print(xTrain01.shape)
xTrain01.head()

(859, 7)


,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age
0,0.000000,0.0,0.321357,0.07764,0.694767,0.205720,0.074176
1,0.000000,0.0,0.321357,0.07764,0.738372,0.205720,0.074176
2,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,0.739011
3,0.396494,0.0,0.848303,0.00000,0.380814,0.000000,1.000000
4,0.368392,0.0,0.560878,0.00000,0.515698,0.580783,0.986264


In [16]:
print(yTrain01.shape)
yTrain01.head()

(859, 1)


,Concrete_compressive_strength
0,0.967445
1,0.741964
2,0.472642
3,0.482400
4,0.522806


# Arquitectura de la red según lo aprendido en clase.

* Funciones de activacion:

    1. Relu como función de activación para la neurona de salida

* Funcion de costo:

    1. MSE.

* Input y Output layer:

    1. Siete parámetros de entrada, siete features.
    2. Una capa de salida por ser regresión.

* Hidden Layer(s):

    1. Dos capas ocultas
    2. Cinco neuronas por capa oculta.
    3. Función de activación Relu por capa.

* Optimizadores:  

    1. Se utilizara el optimizador Nesterov y el Adam.

* Habrá deteniendo el entrenamiento cuando el error comience a aumentar.

    1. Early stopping

* Si hubiera overfitting.

    1. Se utilizará dropout

* Se aplicara mini batch gradient descent con un batch de 64, y a partir de ahi se harán pruebas.

* Como numero de neuronas en cada capa interna, se utilizaran 5 neuronas.

* Como numero de capas internas, el modelo utilizará dos capas.

* Los hiper parametros en base al avance del proyecto



### Creacion del modelo:

### batches:

In [17]:
def batchsizeDivision(x=xTrain01,y=yTrain01,batchSize=128):
  batchesListX = []
  batchesListY = []
  for i in range((x.shape[0]//batchSize)+1):
    if i>=(x.shape[0]/batchSize):
      batchesListX.append(x[i*batchSize:])
      batchesListY.append(y[i*batchSize:])
    else:
      batchesListX.append(x[i*batchSize:(i+1)*batchSize])
      batchesListY.append(y[i*batchSize:(i+1)*batchSize])
  return (batchesListX,batchesListY)

### Grafo:

In [18]:
MLPgraph = tf.Graph()
with MLPgraph.as_default():
  #Carga de la data:
  with tf.name_scope('pureData') as scope:

    #Inicializando matrices de pesos (Xavier inicialization)
    w01 = tf.get_variable(name= "W1", shape=[8, 5],
           initializer=tf.contrib.layers.xavier_initializer())
    w02 = tf.get_variable(name= "W2", shape=[6, 5],
           initializer=tf.contrib.layers.xavier_initializer())
    w03 = tf.get_variable(name= "W3", shape=[6, 1],
            initializer=tf.contrib.layers.xavier_initializer())
    #Aplicando matriz de pesos a capa:
    preX = tf.placeholder(dtype=tf.float32,name='X', shape = [None,7])
    Y = tf.placeholder(dtype=tf.float32,name='Y', shape= [None,1])
    learningRate = tf.placeholder(dtype=tf.float32,name='LR')

    X = tf.pad(tensor=preX,paddings= tf.constant([[0,0],[1,0]]),
               mode='CONSTANT',constant_values=1.0000) #vector de 8x128


  with tf.name_scope('feedFoward') as scope:
    #Aplicando feedFoward - Primera capa oculta
    a01 = tf.nn.relu(tf.matmul(X,w01))
    A01 = tf.pad(tensor=a01,paddings= tf.constant([[0,0],[1,0]]),
               mode='CONSTANT',constant_values=1.0000) 
    #Aplicando feedFoward - Segunda capa oculta
    a02 = tf.nn.relu(tf.matmul(A01,w02))
    A02 = tf.pad(tensor=a02,paddings= tf.constant([[0,0],[1,0]]),
                 mode='CONSTANT',constant_values=1.0000) 
    #Aplicando feedForward - Tercera capa oculta
    a03 = tf.nn.relu(tf.matmul(A02,w03))   
  
  with tf.name_scope('costFunctionMSE') as scope:
    costFunction = 0.5*tf.reduce_mean(tf.pow((Y-a03),2))

  with tf.name_scope('gradient_optimizer') as scope:
    optimizador =  tf.train.AdamOptimizer(learningRate).minimize(costFunction)
    #optimizer = tf.train.AdamOptimizer(learning_rate).minimize(costFunction,var_list=[w01,w02,w03])

### Funcion de entrenamiento:

In [19]:
def trainingFunction(x,y,lr,epoch,showError):
  #Resetteando el grafo
  tf.reset_default_graph()
  #Iniciando la sesion.
  with tf.Session(graph = MLPgraph) as session:
    init=tf.global_variables_initializer().run() 
    #Utilizando mini-batch:
    xBatchList, yBatchList = batchsizeDivision(x,y,128)
    #Cada epoch implica una corrida total de los batch generados.
    for i in range(epoch):
      for b in range(len(xBatchList)):
        MSE,W1,W2, W3,_ = session.run([costFunction, w01,w02,w03,optimizador],
                                      feed_dict = {preX:xBatchList[b],Y:yBatchList[b],learningRate:lr})
      if i%showError == 0:
        print('Epoch: '+str(i)+' error: '+str(MSE))
  return W1,W2,W3

# Experimentacion Cross Validation

In [20]:
W01, W02, W03 = trainingFunction(x=xTrain01,y=yTrain01,lr=0.0001,epoch=100,showError=10)

Epoch: 0 error: 0.06893772
Epoch: 10 error: 0.05930591
Epoch: 20 error: 0.048757646
Epoch: 30 error: 0.04071323
Epoch: 40 error: 0.037013713
Epoch: 50 error: 0.03665647
Epoch: 60 error: 0.037713107
Epoch: 70 error: 0.038791113
Epoch: 80 error: 0.039474342
Epoch: 90 error: 0.039716024


In [21]:
W11, W12, W13 = trainingFunction(x=xTrain02,y=yTrain02,lr=0.0001,epoch=100,showError=10)

Epoch: 0 error: 0.06779906
Epoch: 10 error: 0.051214527
Epoch: 20 error: 0.03676806
Epoch: 30 error: 0.02653249
Epoch: 40 error: 0.01995951
Epoch: 50 error: 0.015789827
Epoch: 60 error: 0.013116302
Epoch: 70 error: 0.01132149
Epoch: 80 error: 0.010044955
Epoch: 90 error: 0.009084688


In [22]:
W21, W22, W23 = trainingFunction(x=xTrain03,y=yTrain03,lr=0.0001,epoch=100,showError=10)

Epoch: 0 error: 0.1496829
Epoch: 10 error: 0.14322972
Epoch: 20 error: 0.1373001
Epoch: 30 error: 0.13181706
Epoch: 40 error: 0.12673095
Epoch: 50 error: 0.12199998
Epoch: 60 error: 0.11762436
Epoch: 70 error: 0.11353858
Epoch: 80 error: 0.10970413
Epoch: 90 error: 0.10608115


# Aplicacion de resultados:

Vemos que al momento de utilizar la data creada utilizando cross-validation, el que mejor desempeño tuvo en el entrenamiento fue el segundo experimento, por tanto, procedemos a utilizarlo con la data real.

In [25]:
print(W11.shape)
print(W12.shape)
print(W13.shape)

(8, 5)
(6, 5)
(6, 1)


In [26]:
def validData(y,x,w1,w2,w3):
    realX = np.append(x,np.ones_like(y),axis=1)
    firstLayer = np.matmul(realX,w1)
    secondLayer = np.matmul(np.append(firstLayer,np.ones_like(y),axis=1),w2)
    outputLayer = np.matmul(np.append(secondLayer,np.ones_like(y),axis=1),w3)
    MSE = 0.5*np.mean(np.power((y-outputLayer),2))
    return MSE
    

### Primeros parámetros de prueba

In [27]:
print(validData(yTrain01,xTrain01,W01, W02, W03))

Concrete_compressive_strength    0.481946
dtype: float64


In [28]:
print(validData(yTrain02,xTrain02,W01, W02, W03))

Concrete_compressive_strength    0.575406
dtype: float64


In [29]:
print(validData(yTrain03,xTrain03,W01, W02, W03))

Concrete_compressive_strength    0.578371
dtype: float64


### Segundos parametros de prueba y los diversos sets de entrenamiento

In [30]:
print(validData(yTrain01,xTrain01,W11, W12, W13))

Concrete_compressive_strength    1.389748
dtype: float64


In [31]:
print(validData(yTrain02,xTrain02,W11, W12, W13))

Concrete_compressive_strength    1.471007
dtype: float64


In [32]:
print(validData(yTrain03,xTrain03,W11, W12, W13))

Concrete_compressive_strength    1.39786
dtype: float64


### Terceros parametros y los diversos sets de entrenamiento

In [33]:
print(validData(yTrain01,xTrain01,W21, W22, W23))

Concrete_compressive_strength    0.491744
dtype: float64


In [34]:
print(validData(yTrain02,xTrain02,W21, W22, W23))

Concrete_compressive_strength    0.511539
dtype: float64


In [35]:
print(validData(yTrain03,xTrain03,W21, W22, W23))

Concrete_compressive_strength    0.550964
dtype: float64


# Comentarios:

Ha quedado demostrado que el segundo modelo da los mejores resultados para la resistencia del cemento, en base a las técnicas y parámetros realizados.